In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install larq
!pip uninstall sklearn
!pip install pretty_confusion_matrix
%mkdir dataset
%cd /content/drive/MyDrive/project
!unrar x /content/drive/MyDrive/project/dataset.rar -d /content/dataset

All OK


In [ ]:
from keras.callbacks import EarlyStopping, ModelCheckpoint, LearningRateScheduler
from keras.layers import Softmax, Dropout, Flatten, MaxPooling2D, BatchNormalization
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model, Sequential
from larq.layers import QuantConv2D
from keras.optimizers import Adam
from keras import backend as K
import tensorflow as tf
import larq as lq
import os
import matplotlib
matplotlib.use('agg')
import matplotlib.pyplot as plt
# 導入資料集
train = ImageDataGenerator(rescale=1/255)
val = ImageDataGenerator(rescale=1/255)
train_dataset = train.flow_from_directory('/content/dataset/train/',
                      target_size=(94,94),
                      batch_size=128,
                      color_mode='grayscale',
                      class_mode='categorical')
val_dataset = val.flow_from_directory('/content/dataset/validation/',
                    target_size=(94,94),
                    batch_size=128,
                    color_mode='grayscale',
                    class_mode='categorical')
kwargs = dict(kernel_quantizer="ste_sign",
        kernel_constraint="weight_clip")
# 定義模型架構
model = Sequential()
model.add(QuantConv2D(filters=16, kernel_size=(3, 3), input_shape=(94, 94, 1), **kwargs))
model.add(MaxPooling2D())
model.add(BatchNormalization(momentum=0.9))
model.add(QuantConv2D(filters=32, kernel_size=(3, 3), input_quantizer="ste_sign", **kwargs))
model.add(MaxPooling2D())
model.add(BatchNormalization(momentum=0.9))
model.add(QuantConv2D(filters=64, kernel_size=(3, 3), input_quantizer="ste_sign", **kwargs))
model.add(MaxPooling2D())
model.add(BatchNormalization(momentum=0.9))
model.add(QuantConv2D(filters=64, kernel_size=(3, 3), input_quantizer="ste_sign", **kwargs))
model.add(MaxPooling2D())
model.add(BatchNormalization(momentum=0.9))
model.add(QuantConv2D(filters=64, kernel_size=(3, 3), input_quantizer="ste_sign", **kwargs))
model.add(MaxPooling2D())
model.add(BatchNormalization(momentum=0.9))
model.add(QuantConv2D(filters=32, kernel_size=(1, 1), input_quantizer="ste_sign", **kwargs))
model.add(BatchNormalization(momentum=0.9))
model.add(QuantConv2D(filters=10, kernel_size=(1, 1), input_quantizer="ste_sign", activation='softmax', **kwargs))
model.add(Flatten())

adam = Adam(learning_rate=0.01)
model.compile(optimizer=adam,
       loss='categorical_crossentropy',
       metrics=['accuracy'])
# 定義學習率調整函數
def lr_scheduler(epoch, learning_rate):
    if epoch > 0 and epoch % 20 ==0:
        return learning_rate * 0.5
    else:
        return learning_rate

# 創建學習率調度器
lr_callback = LearningRateScheduler(lr_scheduler)
# 早停機制
earlystop_callback = EarlyStopping(monitor='val_accuracy', min_delta=0,
                  patience=15, mode='auto')
# 斷點續訓
checkpoint_callback = ModelCheckpoint('checkpoint.ckpt', monitor='val_accuracy',
                    save_weights_only=True, verbose=1,
                    save_best_only=True, mode='auto')
'''
if os.path.exists('checkpoint'):
    model.load_weights('checkpoint.ckpt')
    print("Cotinue with checkpoint")
else:
    print("No checkpoint")
'''
print('Training ---------------------------------')
history = model.fit(train_dataset,
           epochs=100,
           batch_size=128,
           validation_data=val_dataset,
           callbacks=[lr_callback,
                earlystop_callback,
                checkpoint_callback])
model.save('model.h5')

# 繪製圖表
plt.figure()
loss = history.history['loss']
val_loss = history.history['val_loss']
acc = history.history['accuracy']
val_acc= history.history['val_accuracy']
plt.subplot(2, 1, 1)
plt.title('model loss')
plt.plot(loss, label='loss')
plt.plot(val_loss, label='val_loss')
plt.ylabel('value')
plt.xlabel('epoch')
plt.legend(['train_loss',
      'valid_loss'],
      loc='upper right')
plt.subplot(2, 1, 2)
plt.title('model accuracy')
plt.plot(acc, label='acc')
plt.plot(val_acc, label='val_acc')
plt.ylabel('value')
plt.xlabel('epoch')
plt.legend(['train_accuracy',
      'valid_accuracy'],
      loc='upper right')
plt.tight_layout()
plt.savefig('./result.png')


Found 5000 images belonging to 10 classes.
Found 1000 images belonging to 10 classes.
Training ---------------------------------
Epoch 1/100
40/40 [==============================] - ETA: 0s - loss: 1.8435 - accuracy: 0.6660
Epoch 1: val_accuracy improved from -inf to 0.72600, saving model to checkpoint.ckpt
40/40 [==============================] - 30s 178ms/step - loss: 1.8435 - accuracy: 0.6660 - val_loss: 1.3162 - val_accuracy: 0.7260 - lr: 0.0100
Epoch 2/100
40/40 [==============================] - ETA: 0s - loss: 0.3312 - accuracy: 0.9164
Epoch 2: val_accuracy improved from 0.72600 to 0.77600, saving model to checkpoint.ckpt
40/40 [==============================] - 8s 196ms/step - loss: 0.3312 - accuracy: 0.9164 - val_loss: 1.6250 - val_accuracy: 0.7760 - lr: 0.0100
Epoch 3/100
40/40 [==============================] - ETA: 0s - loss: 0.0992 - accuracy: 0.9720
Epoch 3: val_accuracy improved from 0.77600 to 0.86500, saving model to checkpoint.ckpt
40/40 [=============================

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
from keras.preprocessing.image import ImageDataGenerator
from pretty_confusion_matrix import pp_matrix_from_data
from keras.models import load_model
import numpy as np
import cv2
import matplotlib
matplotlib.use('agg')
import matplotlib.pyplot as plt

test = ImageDataGenerator(rescale=1/255)
test_dataset = test.flow_from_directory('/content/dataset/test/',
                     target_size=(94,94),
                     batch_size=256,
                     color_mode='grayscale',
                     class_mode='categorical',
                     shuffle=False)

model = load_model('model.h5')
print('\nTesting ------------')
loss, accuracy = model.evaluate(test_dataset)
print('\ntest loss: ', loss)
print('\ntest accuracy: ', accuracy)

j=1
k=0
for i in 'down','left','ok','paper','right','rock','scissors','stone','thumb','up':
    x = cv2.imread('/content/dataset/test/{}/{}.jpg'.format(i,j))
    x = cv2.cvtColor(x, cv2.COLOR_BGR2GRAY)
    x = x/255
    x = cv2.resize(x, (94, 94))
    x = np.expand_dims(x, axis=0)
    pred = model.predict(x)
    #print(pred)
    print(np.argmax(pred)+1)
    if np.argmax(pred)+1 == j:
      print('correct')
      k+=1
    else:
      print('false')
    j+=1
print(k)


# 預測測試資料
Y_pred = model.predict(test_dataset)
# 將預測結果轉成類別編號
y_pred = np.argmax(Y_pred, axis=1)
# 取得真實類別編號
y_true = test_dataset.classes
# 繪製混淆矩陣
class_name = ['down',
        'left',
        'ok',
        'paper',
        'right',
        'rock',
        'scissors',
        'stone',
        'thumb',
        'up']
cmap = 'terrain'
pp_matrix_from_data(y_true, y_pred, cmap=cmap, columns=class_name)
plt.savefig('./confusion.png')


Found 1000 images belonging to 10 classes.

Testing ------------
4/4 [==============================] - 2s 230ms/step - loss: 0.0994 - accuracy: 0.9750

test loss:  0.09938722848892212

test accuracy:  0.9750000238418579
1/1 [==============================] - 0s 403ms/step
1
correct
1/1 [==============================] - 0s 18ms/step
2
correct
1/1 [==============================] - 0s 17ms/step
3
correct
1/1 [==============================] - 0s 18ms/step
4
correct
1/1 [==============================] - 0s 20ms/step
5
correct
1/1 [==============================] - 0s 20ms/step
6
correct
1/1 [==============================] - 0s 21ms/step
7
correct
1/1 [==============================] - 0s 19ms/step
8
correct
1/1 [==============================] - 0s 23ms/step
9
correct
1/1 [==============================] - 0s 20ms/step
10
correct
10
4/4 [==============================] - 1s 244ms/step


In [ ]:
with lq.context.quantized_scope(True):
   model.set_weights(model.get_weights())

for layer in model.layers:
  if 'conv' in layer.name:
    print(layer.name)
    weight, bias = model.get_layer(layer.name).get_weights()
    print(np.shape(weight))
    f = open('./weight/W_'+layer.name+'.bin', 'w')
    for i in range(np.shape(weight)[0]):
      for j in range(np.shape(weight)[1]):
        for k in range(np.shape(weight)[2]):
          for l in range(np.shape(weight)[3]):
            f.write(str(weight[i][j][k][l])+"\n")
    f.close()
    f = open('./weight/b_'+layer.name+'.bin', 'w')
    for i in range(np.shape(bias)[0]):
      f.write(str(bias[i])+"\n")
    f.close()

for layer in model.layers:
  if 'batch_normalization' in layer.name:
    print(layer.name)
    a = model.get_layer(layer.name).get_weights()
    weight = a[0]/pow(a[3]+0.001, 0.5)
    bias = -(a[0]*a[2])/pow(a[3]+0.001, 0.5) + a[1]
    print(np.shape(weight))
    f = open('./weight/W_'+layer.name+'.bin', 'w')
    for i in range(np.shape(weight)[0]):
      f.write(str(weight[i])+"\n")
    f.close()
    f = open('./weight/b_'+layer.name+'.bin', 'w')
    for i in range(np.shape(bias)[0]):
      f.write(str(bias[i])+"\n")
    f.close()

quant_conv2d
(3, 3, 1, 16)
quant_conv2d_1
(3, 3, 16, 32)
quant_conv2d_2
(3, 3, 32, 64)
quant_conv2d_3
(3, 3, 64, 64)
quant_conv2d_4
(3, 3, 64, 64)
quant_conv2d_5
(1, 1, 64, 32)
quant_conv2d_6
(1, 1, 32, 10)
batch_normalization
(16,)
batch_normalization_1
(32,)
batch_normalization_2
(64,)
batch_normalization_3
(64,)
batch_normalization_4
(64,)
batch_normalization_5
(32,)


In [ ]:
for layer in model.layers:
  if 'conv' in layer.name:
    weight, bias = model.get_layer(layer.name).get_weights()
    print(layer.name, weight.shape, bias.shape)

quant_conv2d (3, 3, 1, 16) (16,)
quant_conv2d_1 (3, 3, 16, 32) (32,)
quant_conv2d_2 (3, 3, 32, 64) (64,)
quant_conv2d_3 (3, 3, 64, 64) (64,)
quant_conv2d_4 (3, 3, 64, 64) (64,)
quant_conv2d_5 (1, 1, 64, 32) (32,)
quant_conv2d_6 (1, 1, 32, 10) (10,)


In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 quant_conv2d (QuantConv2D)  (None, 92, 92, 16)        160       
                                                                 
 max_pooling2d (MaxPooling2  (None, 46, 46, 16)        0         
 D)                                                              
                                                                 
 batch_normalization (Batch  (None, 46, 46, 16)        64        
 Normalization)                                                  
                                                                 
 quant_conv2d_1 (QuantConv2  (None, 44, 44, 32)        4640      
 D)                                                              
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 22, 22, 32)        0         
 g2D)                                                   

In [ ]:
import cv2
import larq as lq
import numpy as np
from keras.models import Model
from keras.models import load_model
model = load_model('model.h5')
x = cv2.imread('/content/dataset/test/down/1.jpg')
x = cv2.cvtColor(x, cv2.COLOR_BGR2GRAY)
x = x/255
x = cv2.resize(x, (94, 94))
x = np.expand_dims(x, axis=0)
conv_model = Model(inputs=model.input, outputs=model.get_layer('quant_conv2d').output)
conv_output = conv_model.predict(x)
print(conv_output)


1/1 [==============================] - 0s 112ms/step
[[[[ 1.8907471  -0.63083786  0.57249606 ... -1.8391258   1.8481421
    -3.0651014 ]
   [ 1.8814781  -0.6447635   0.61004615 ... -1.9223722   1.8997114
    -3.0990105 ]
   [ 1.8612368  -0.6475956   0.56575626 ... -1.841873    1.8192127
    -3.123893  ]
   ...
   [ 1.9866748  -0.661149    0.62287825 ... -1.9642822   1.949538
    -3.256819  ]
   [ 1.9833735  -0.6723746   0.599552   ... -1.969585    1.933385
    -3.277365  ]
   [ 2.0170527  -0.6964586   0.6212542  ... -1.9650124   1.9778283
    -3.2726817 ]]

  [[ 1.8894446  -0.65010613  0.61160564 ... -1.8744434   1.836853
    -3.1475296 ]
   [ 1.858901   -0.6743751   0.5842354  ... -1.805817    1.8204107
    -3.1072593 ]
   [ 1.8801234  -0.6174872   0.57951057 ... -1.8265129   1.8172901
    -3.0786185 ]
   ...
   [ 1.983644   -0.66658163  0.6105025  ... -1.9592508   1.9515553
    -3.264004  ]
   [ 1.990148   -0.67151636  0.6118298  ... -1.9676757   1.9741397
    -3.2881107 ]
   [ 2.004

In [ ]:
pool_model = Model(inputs=model.input, outputs=model.get_layer('max_pooling2d').output)
pool_output = pool_model.predict(x)
print(pool_output)

1/1 [==============================] - 1s 516ms/step
[[[[ 1.8907471  -0.63083786  0.61160564 ... -1.805817    1.8997114
    -3.0651014 ]
   [ 1.8801234  -0.6174872   0.57951057 ... -1.8265129   1.8711238
    -3.0666397 ]
   [ 1.9135083  -0.59297323  0.6110509  ... -1.8408482   1.8677378
    -3.0844448 ]
   ...
   [ 2.0189285  -0.67019874  0.6615631  ... -1.9483994   2.0006843
    -3.258609  ]
   [ 1.9866748  -0.661149    0.62287825 ... -1.9306251   1.9572618
    -3.2498574 ]
   [ 2.0170527  -0.67151636  0.6212542  ... -1.9650124   1.9808464
    -3.2726817 ]]

  [[ 1.890965   -0.5990975   0.5975708  ... -1.8456496   1.8711495
    -3.0329056 ]
   [ 1.8847841  -0.60490215  0.5979783  ... -1.8399743   1.8788111
    -3.0760815 ]
   [ 1.8926609  -0.60498285  0.6168919  ... -1.819249    1.8958147
    -3.0692992 ]
   ...
   [ 2.0113807  -0.64245874  0.6354794  ... -1.9318477   2.0088606
    -3.2305903 ]
   [ 2.0015535  -0.65165526  0.61340314 ... -1.9387602   1.989221
    -3.2651298 ]
   [ 2.0

In [ ]:
bn_model = Model(inputs=model.input, outputs=model.get_layer('batch_normalization').output)
bn_output = bn_model.predict(x)
print(bn_output)

1/1 [==============================] - 1s 539ms/step
[[[[ 0.29873782 -0.7511627   0.14810602 ... -0.2458733   0.7354265
    -0.547823  ]
   [ 0.28033894 -0.70645255  0.03431716 ... -0.28026688  0.67822516
    -0.54924655]
   [ 0.3381573  -0.6243575   0.14613926 ... -0.30409     0.67144996
    -0.5657241 ]
   ...
   [ 0.5207314  -0.8829787   0.3252236  ... -0.4828244   0.9374647
    -0.726902  ]
   [ 0.4648721  -0.85267204  0.18807156 ... -0.45328602  0.85058004
    -0.7188029 ]
   [ 0.51748264 -0.88739127  0.18231373 ... -0.5104328   0.89777076
    -0.7399254 ]]

  [[ 0.2991152  -0.6448671   0.09834725 ... -0.31206924  0.67827666
    -0.51802784]
   [ 0.28841072 -0.6643064   0.09979204 ... -0.30263767  0.69360685
    -0.5579844 ]
   [ 0.30205226 -0.66457665  0.16684781 ... -0.26819533  0.7276295
    -0.5517078 ]
   ...
   [ 0.50765944 -0.79007995  0.23274724 ... -0.45531783  0.9538248
    -0.7009725 ]
   [ 0.49064013 -0.82087827  0.1544788  ... -0.46680534  0.91452765
    -0.7329365 ]


In [ ]:
a = model.get_layer('batch_normalization').get_weights()
weight = a[0]/pow(a[3]+0.001, 0.5)
bias = -(a[0]*a[2])/pow(a[3]+0.001, 0.5) + a[1]
print(weight)
print(bias)

[1.7318696  3.3489084  3.5453682  1.3990011  4.540227   3.2058363
 3.0234506  1.1260498  4.230028   1.6504177  2.6673472  1.5945413
 0.97769845 1.6618536  2.0009155  0.9254362 ]
[-2.9757895   1.3614556  -2.020261    1.8797262  -2.7428293   1.5841062
  1.3130219  -3.3995714  -2.5005271   2.459118   -1.5865941   2.2283335
  0.79317385  2.75513    -3.0657353   2.2887325 ]
